# It gets loopy

In this notebook, we will work with stock prices. I'll get you started by writing some code that go and fetch the last quotes of a few American stocks. We will then write loops and functions to do two things:

1. Evaluate the performance of an investing strategy on past data
2. Infer the possible performance of the same strategy on short-run future data

## Setup

We start by importing the quotes. It is not crucial to understand the code here below as we will see what a call to an API is during a future session. It works broadly like this: we call a server that sends us back a kind of nested dictionaries and lists that contain the quotes and date of those quotes for the 5 largest tech firms (that are sometimes referred to as GAFAM).

In [ ]:
from requests import get
stock_quotes = {}
tickers = {"GOOG" : "Google", "AAPL" : "Apple", "FB": "Facebook", "AMZN" : "Amazon", "MSFT": "Microsoft"}
for ticker in tickers:
    prices_and_dates = {}
    query = "https://financialmodelingprep.com/api/v3/historical-price-full/"+ticker+"?serietype=line"
    histoire_des_prix = get(query).json()["historical"]
    prices_and_dates["quotes"] = [cours["close"] for cours in histoire_des_prix[-31:]]
    prices_and_dates["dates"] = [cours["date"] for cours in histoire_des_prix[-31:]]
    stock_quotes[ticker] = prices_and_dates

Run the cell above: it will run for some time and then ends without showing anything. In fact, it loads a peculiar data structure in the memory of your computer. The structure consists in prices and dates contained inside a dictionary which is itself contained in a dictionary. It is kind of a lousy structure so far but we will see in next sessions (the pandas one) how to make it easier. For now, we have to rely on this awkward structure. Here are some example of how to use it.

In [ ]:
# Show all prices for Google
print(stock_quotes["GOOG"]["quotes"])

In [ ]:
# Show the price of the Apple closing quote yesterday
print(stock_quotes["AAPL"]["quotes"][-1])

In [ ]:
# What is the date of the third quote for the Microsoft data?
print(stock_quotes["MSFT"]["dates"][2])

## Let's work

OK, enough playing around! Now is your turn: Let's start by stretching our programming muscles. I'll write a line that will create a new variable called *quotes_amazon* and that is a list that contains the quotes of Amazon (in dollars) for all the data we have.

Your job is to create a new list, called *quotes_amazon_euro* in which you will store the quotes but in Euros (for this exercise, it is enough to consider that converting to Euros just means multiplying by *0.9* - please applied trade economists, don't crucify me for this approximation).

In [ ]:
quotes_amazon = stock_quotes["AMZN"]["quotes"]

# Your code comes below this line
print([quote * 0.9 for quote in quotes_amazon])

Done? Good! Now for a harder problem: We will create a function that will compute the mean of any list that is passed (provided it contains integers or floats, of course). The function will work broadly like this: You initialize a counter at 0, you loop over the list and, at each iteration, you add the value at that position to the counter. Once the loop is over, you divide this by the length of the list to obtain the average.

Oddly enough, Python doesn't ship with a built-in mean function (it was not initially a scientific computing language). Implementing it is therefore needed to perform the next steps (in future session, we will see some libraries, the most important being Numpy, that provide implementation of means functions.

Implement your function, call it my_mean and then see if it works on the list containing the prices of the Facebook stock (that will be stored in the variable *facebook_quotes*.

*Don't forget that you must produce a **function**, it will be important because we will use it in the next part!*

In [ ]:
import numpy as np
facebook_quotes = stock_quotes["FB"]["quotes"]
print("Your function should produce the following result (plus ou minus numerical error):\t"
      , np.mean(facebook_quotes))

# Your code comes here below
def mean(numbers_to_average):
    return sum(numbers_to_average)/len(numbers_to_average)

print(mean(facebook_quotes))

Let's now build on top of what we did previously to build a function that computes the mobile average. Let's start by using a fix number of lags to be considered (you can always extend it later): We want to have a function that takes a list and produces a new list containing the average value of the three past value (you can either align the two list by setting the three first value of your result to *None* or not, this will impact the way you will have to solve the next exercises, but both ways will be equally practical). Test it on the Google quotes that are in the variable *google_quotes*.

In [ ]:
google_quotes = stock_quotes["GOOG"]["quotes"]

# Your code comes here below
def moving_average_3(list_to_treat):
    result = [None, None, None]
    for i in range(3,len(list_to_treat)):
        result.append(round(mean(list_to_treat[i-3:i]),2))
    return result
        
print(moving_average_3(google_quotes))

Now that we have done it for a fix number of lags, do it again but now allow for a second parameter to your function: the number of lags you want to consider for your moving average.

In [ ]:
def moving_average(list_to_treat, order):
    results = [None for i in range(order)]
    for i in range(order, len(list_to_treat)):
        results.append(round(mean(list_to_treat[i-order:i]),2))
    return results

print(moving_average(google_quotes, 4))

We are now going to compute the mean squared error. Write a function that sums over the squared differences between the actual value and your moving-average-based forecast. This will help us determining the optimal number of lags for our secret investment strategy at the next step.

In [ ]:
def mse(list_1, list_2):
    if len(list_1) != len(list_2):
        print("Both list must be of the same size")
        return None
    accumulator = 0
    count = 0
    for i in range(len(list_1)):
        if list_1[i] != None and list_2[i] != None:
            accumulator += (list_1[i] - list_2[i])**2
            count += 1
    return accumulator/count

print(mse([1, 2, 2, 5], [None, None, 4, 3]))

Our masterplan to get rich is getting together nicely...

![coming_together](https://i1.wp.com/media1.tenor.com/images/a71c94c3aa7ad66a5051f81f48d14dd2/tenor.gif?w=688&ssl=1)

The last step before creating the actual investement strategy: we are going to evaluate which lag produces the smallest mean square error. For that, create a function that loops over the range between 2 and 6 and return the optimal lag (the one with the smallest RMSE).

In [ ]:
def optimal_order(a_list):
    optimum = 0
    minimum_mse = 1_000_000_000_000 # that's a comically large value
    for i in range(2, 7):
        MA = moving_average(a_list, i)
        mse_for_this_lag = mse(a_list, MA)
        if mse_for_this_lag < minimum_mse:
            minimum_mse = mse_for_this_lag
            optimum = i
    return optimum

print(optimal_order(google_quotes))

We are now going to devise an incredibly dumb investment strategy: if your moving-average prediction (with the optimal lag - see, we are already [double-dipping](https://en.wikipedia.org/wiki/Circular_analysis) in the data...) is higher than the current price on a day, we invest all our accrued fortune (if we currently had no stock) or hold (if we had already some value invested), if the prediction is below, we sell our whole position (or do nothing if we had no money invested at the time).

How much would we have today if we had started using this strategy on February 3rd and started out with 100 Euros?

This is a significantly more complex problem than the ones above and you might want to split it into subproblems (for example, a good idea could be to devise a function that takes two values and returns the variation between those). There are many ways to solve it, find the one that makes more sense to you. The result should be the same for everyone, though.

In [ ]:
compute_variation = lambda day_before, current_day : round((current_day - day_before)/day_before, 4)

update_value = lambda current_value, variation : current_value * (1 + variation)

def choose_investment_decision(value_today, forecast_tomorrow, amount_invested_today, amount_in_cash_today):
    new_amount_in_cash, new_amount_in_stocks = amount_in_cash_today, amount_invested_today # the baseline is that we keep the situation as is
    if value_today >= forecast_tomorrow: # the stock is predicted to loose money
        if amount_in_cash_today == 0: # We had money on the stock
            new_amount_in_cash = amount_invested_today
            new_amount_in_stocks = amount_in_cash_today
    elif value_today < forecast_tomorrow: # The stock is predicted to make money
        if amount_in_cash_today != 0: # Our money was in cash
            new_amount_in_cash = amount_invested_today
            new_amount_in_stocks = amount_in_cash_today
    # Note: we can convert this large if by removing the elif and using boolean algebra (because the body is the same)
    return new_amount_in_cash, new_amount_in_stocks

def index_date(date_list, date):
    for i in range(len(date_list)): 
        if date_list[i] == date:
            return i
    return -1

def simulate_investment(stock, start_simulation, start_cash, end_simulation = -1):
    if end_simulation == -1: end_simulation = len(stock)-1
    cash_stocks = (start_cash, 0)
    optimal_lag = optimal_order(stock)
    forecast = moving_average(stock, optimal_lag)
    for ii in range(start_simulation, end_simulation):
        variation = compute_variation(stock[ii], stock[ii-1])
        cash_stocks = (cash_stocks[0], update_value(cash_stocks[1], variation)) # The value of the stocks are updated
        cash_stocks = choose_investment_decision(stock[ii], forecast[ii+1], cash_stocks[1], cash_stocks[0])
    return sum(cash_stocks)

stock_to_analyse = stock_quotes["AAPL"]
index_start = index_date(stock_to_analyse["dates"], '2020-02-03')
simulate_investment(stock_to_analyse["quotes"], index_start, 100)

## Get rich (fast) or code tryin'

The last step for today is to assess how much we will make in the near future. For this, we will use a small, non-parametric Monte-Carlo simulation.

![monte_carlo](http://www.azquotes.com/picture-quotes/quote-in-mathematics-as-in-physics-so-much-depends-on-chance-on-a-propitious-moment-stanislaw-ulam-111-21-35.jpg)

Here's how it will go: we are going to reuse the function that simulates the performance of our stupid investment strategy but on simulated data that will be produced in the following way: we are going to simulate 20 periods of data by resampling at random the past prices. This can be done using the function [random.choices(population, weights=None, \*, cum_weights=None, k=1)](https://docs.python.org/3/library/random.html#random.choices): We are going to produce a list containing all the price variations of the quote. We then are going to produce a list of 20 values between 0 and the length of the list we just create and use those as variations over the next 20 days. We test our investment strategy again this "future reality" as if we started today with 100 Euros and the result is the final value we would have after 20 days. We have one potential result that we store in a list, called *results*. We repeat the process 5000 times (you can do it less time if your computer is rather slow). In the end, we will end up with 5000 potential results of our investment strategy over the 20 next days. I'll write the function that plots the distribution so that we can see visually what we can expect.

Again, this whole procedure will probably require several functions. Try to break down the problem in a sequence of intermediate steps.

In [ ]:
import random

def produce_variations_lists(stock_prices):
    variations = []
    for ii in range(1,len(stock_prices)):
        variations.append(compute_variation(stock_prices[ii - 1], stock_prices[ii]))
    return variations

def produce_simulation(list_prices, n_future_periods):
    return random.choices(produce_variations_lists(list_prices), k=n_future_periods)

def simulate_profit(stock_prices, n_future_steps, n_monte_carlo):
    results = []
    for i in range(n_monte_carlo):
        variations = produce_simulation(stock_prices, n_future_steps)
        length_original_list = len(stock_prices)
        simulated_list = stock_prices[:]
        for variation_that_day in variations:
            simulated_list.append(simulated_list[-1]*(1 + variation_that_day))
        results.append(simulate_investment(simulated_list, length_original_list, 100))
    return results

stock_to_analyse = stock_quotes["AAPL"]["quotes"][:]
results = simulate_profit(stock_to_analyse, 20, 5000)

The next cell will represent the distribution of your profits, only execute it once you have your 5000 results in the variable *results*.

In [ ]:
import seaborn as sns

sns.distplot(results)